In [15]:
df = pd.read_parquet(data_files, engine='pyarrow')
df = df.sample(n=200, random_state=0)
df

,file_name,label,image_data
1573,images/image_1608_19351942.jpg,1942,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
4623,images/image_5211_1581.jpg,1581,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
3315,images/image_3350_c 1940.jpg,1940,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
4431,images/image_4845_1849.jpg,1849,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
247,images/image_260_c 1505.jpg,1505,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
...,...,...,...
3414,images/image_3449_19351942.jpg,1942,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
4551,images/image_5032_probably c 16451656.jpg,1656,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
5883,images/image_6998_possibly c 1530.jpg,1530,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
5670,images/image_6681_c 1400.jpg,1400,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...


In [ ]:
import requests
from transformers import Blip2Processor, Blip2ForConditionalGeneration
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
# load pipelines
blip_processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
blip_model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-opt-2.7b",
    torch_dtype=torch.float16,
    cache_dir='cache',
).to(device)

## IMAGE CPATIONING ##
def caption_images(input_image):
    inputs = blip_processor(images=input_image, return_tensors="pt").to(device, torch.float16)
    pixel_values = inputs.pixel_values

    generated_ids = blip_model.generate(pixel_values=pixel_values, max_length=50)
    generated_caption = blip_processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return generated_caption


In [ ]:
#@title Configs and Mount drive
from google.colab import drive
import os

%cd /content

if not os.path.exists('/content/drive'):
  drive.mount('/content/drive/MyDrive')


In [ ]:
import json
from PIL import Image
import os
import io
# saves each caption and corresponding image to a metadata.jsonl file
local_dir='/content/drive/MyDrive/artwork_dataset/'
if not os.path.exists(local_dir):
    os.makedirs(local_dir)
from tqdm import tqdm
paths_and_images = list(df[["file_name", "image_data"]].itertuples(index=False, name=None))
with open(f'{local_dir}metadata.jsonl', 'w') as outfile:
  for path, img in tqdm(paths_and_images):
      file_name = os.path.basename(path)
      img = Image.open(io.BytesIO(img))
      img.save(f"{local_dir}{file_name}")
      blip_caption=caption_images(img).strip()
      entry = {"file_name":file_name, "prompt": blip_caption}
      json.dump(entry, outfile)
      outfile.write('\n')

print(f"\nYour image captions are ready here: ./metadata.jsonl")

In [17]:
import json

# Specify the path to the metadata.jsonl file
metadata_file = '../metadata.jsonl'

# Specify the path to the images directory
images_directory = './images/'

# Read the metadata.jsonl file
with open(metadata_file, 'r') as f:
    lines = f.readlines()

# Modify the file_name column and save the changes
with open(metadata_file, 'w') as f:
    for line in lines:
        data = json.loads(line)
        data['file_name'] = f"{images_directory}{data['file_name']}"
        f.write(json.dumps(data) + '\n')

In [1]:
!git lfs install

Updated Git hooks.
Git LFS initialized.


In [3]:
!git clone https://huggingface.co/datasets/wintercoming6/artwork_for_sdxl

Cloning into 'artwork_for_sdxl'...
remote: Enumerating objects: 396, done.
remote: Counting objects: 100% (392/392), done.
remote: Compressing objects: 100% (391/391), done.
remote: Total 396 (delta 87), reused 0 (delta 0), pack-reused 4
Receiving objects: 100% (396/396), 91.76 KiB | 11.47 MiB/s, done.
Resolving deltas: 100% (87/87), done.


In [5]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.9 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 2.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 5.7 MB/s eta 0:00:00:00:01


In [6]:
!datasets-cli test ./artwork_for_sdxl/artwork_for_sdxl.py --save_info --all_configs

/usr/local/python/3.10.13/lib/python3.10/site-packages/datasets/load.py:926: FutureWarning: The repository for artwork_for_sdxl contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at ./artwork_for_sdxl/artwork_for_sdxl.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Loading Dataset Infos from /home/codespace/.cache/huggingface/modules/datasets_modules/datasets/artwork_for_sdxl/a572732b58430944a8a7c583ce5987f7eb9e7cd5986709f8ec1d881cbfadadbd
Testing builder 'default' (1/1)
Generating dataset artwork_for_sdxl (/home/codespace/.cache/huggingface/datasets/artwork_for_sdxl/default/0.0.0)
https://huggingface.co/datasets/wintercoming6/artwork_for_sdxl/resolve/main/metadata.jsonl not found in cache or force_download set to True, downloading to /home/codespace/.c

In [9]:
from datasets import load_dataset

# 假设 your_arrow_file.arrow 是你的 Arrow 文件路径
dataset = load_dataset("arrow", data_files="./artwork_for_sdxl/artwork_for_sdxl-train.arrow")

# 现在你可以像使用其他 datasets 数据集一样使用它
print(dataset)


Generating train split: 41 examples [00:00, 11402.10 examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'image_data'],
        num_rows: 41
    })
})
